In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Tasne\\Documents\\Ashter Katkoota\\NLP Projects\\TextSummarizer\\Text_Summarizer_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Tasne\\Documents\\Ashter Katkoota\\NLP Projects\\TextSummarizer\\Text_Summarizer_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path 
    data_path: Path
    model_path: Path 
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath) #the output : ConfigBox({'artifacts_root': 'artifacts'
        # ,'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/samsumdata.zip'
        # , 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}})

        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )

        return model_evaluation_config
    

In [7]:
!pip install evaluate


In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset, load_from_disk
from evaluate import load
from tqdm import tqdm
import torch
import pandas as pd


In [9]:
device = "cpu"


In [10]:
print(type(model))


NameError: name 'model' is not defined

In [16]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]
    
    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer,
                                batch_size=16, device=device,
                                column_text="article",
                                column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))
        
        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device),
                            length_penalty=0.8, num_beams=8, max_length=128)


            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)
                                    for s in summaries]


            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        score = metric.compute()
        return score
    
    def evaluate(self):
        device = "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        # loading data:
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = load("rouge")
        
        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt["test"][0:10], rouge_metric, model_pegasus, tokenizer, 2, column_text="dialogue", column_summary="summary")
        
        rouge_dict = {rn: score[rn] for rn in rouge_names}


        df = pd.DataFrame(rouge_dict, index=["pegasus"])
        df.to_csv(self.config.metric_file_name, index=False)

In [17]:
try:
    config= ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2025-11-03 19:20:35,272: INFO: common: Created directory at artifacts]
[2025-11-03 19:20:35,273: INFO: common: Created directory at artifacts/model_evaluation]
[2025-11-03 19:21:33,274: WARNING: loading: Using the latest cached version of the module from C:\Users\Tasne\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--rouge\b01e0accf3bd6dd24839b769a5fda24e14995071570870922c71970b3a6ed886 (last modified on Mon Nov  3 17:56:48 2025) since it couldn't be found locally at evaluate-metric--rouge, or remotely on the Hugging Face Hub.]


100%|██████████| 5/5 [02:55<00:00, 35.15s/it]

[2025-11-03 19:24:29,056: INFO: rouge_scorer: Using default tokenizer.]
